In [40]:
import pandas as pd, numpy as np, datetime as dt, re
import matplotlib.pyplot as plt, scipy.stats as stats
import sqlalchemy as sql

In [4]:
engine = sql.create_engine(f"sqlite:///../data/interim/NBA.db")
conn = engine.connect()
df_all = pd.read_sql("SELECT * FROM basic_stats", con=conn)

In [5]:
df_all.shape

(830021, 26)

In [8]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 830021 entries, 0 to 830020
Data columns (total 26 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              830021 non-null  int64  
 1   name            830021 non-null  object 
 2   minutes_played  640649 non-null  float64
 3   fg              640649 non-null  float64
 4   fga             640649 non-null  float64
 5   fg_pct          640649 non-null  float64
 6   fg3             640649 non-null  float64
 7   fg3a            640649 non-null  float64
 8   fg3_pct         640649 non-null  float64
 9   ft              640649 non-null  float64
 10  fta             640649 non-null  float64
 11  ft_pct          640649 non-null  float64
 12  orb             640649 non-null  float64
 13  drb             640649 non-null  float64
 14  trb             640649 non-null  float64
 15  ast             640649 non-null  float64
 16  stl             640649 non-null  float64
 17  blk       

In [10]:
df_all.groupby("timetype").count()

,id,name,minutes_played,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,...,ast,stl,blk,tov,pf,pts,bpm,dnp,team,game_code
timetype,,,,,,,,,,,,,,,,,,,,,
1H,117497,117497,92342,92342,92342,92342,92342,92342,92342,92342,...,92342,92342,92342,92342,92342,92342,92342,117497,117497,117497
1Q,117497,117497,87255,87255,87255,87255,87255,87255,87255,87255,...,87255,87255,87255,87255,87255,87255,87255,117497,117497,117497
2H,117497,117497,96168,96168,96168,96168,96168,96168,96168,96168,...,96168,96168,96168,96168,96168,96168,96168,117497,117497,117497
2OT,821,821,428,428,428,428,428,428,428,428,...,428,428,428,428,428,428,428,821,821,821
2Q,117497,117497,89137,89137,89137,89137,89137,89137,89137,89137,...,89137,89137,89137,89137,89137,89137,89137,117497,117497,117497
3OT,133,133,73,73,73,73,73,73,73,73,...,73,73,73,73,73,73,73,133,133,133
3Q,117497,117497,85652,85652,85652,85652,85652,85652,85652,85652,...,85652,85652,85652,85652,85652,85652,85652,117497,117497,117497
4OT,54,54,30,30,30,30,30,30,30,30,...,30,30,30,30,30,30,30,54,54,54
4Q,117497,117497,86246,86246,86246,86246,86246,86246,86246,86246,...,86246,86246,86246,86246,86246,86246,86246,117497,117497,117497


In [14]:
df_all.groupby("name").count().sort_values("fg", ascending=False)

,id,minutes_played,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,fta,...,stl,blk,tov,pf,pts,bpm,dnp,timetype,team,game_code
name,,,,,,,,,,,,,,,,,,,,,
Team Totals,60836,60836,60836,60836,60836,60836,60836,60836,60836,60836,...,60836,60836,60836,60836,60836,60836,60836,60836,60836,60836
Joe Ingles,2181,2151,2151,2151,2151,2151,2151,2151,2151,2151,...,2151,2151,2151,2151,2151,2151,2181,2181,2181,2181
P.J. Tucker,2193,2138,2138,2138,2138,2138,2138,2138,2138,2138,...,2138,2138,2138,2138,2138,2138,2193,2193,2193,2193
Jae Crowder,2254,2119,2119,2119,2119,2119,2119,2119,2119,2119,...,2119,2119,2119,2119,2119,2119,2254,2254,2254,2254
Donovan Mitchell,2138,2092,2092,2092,2092,2092,2092,2092,2092,2092,...,2092,2092,2092,2092,2092,2092,2138,2138,2138,2138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Sheldon Mac,15,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,15,15,15,15
Jaren Jackson Jr.,17,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,17,17,17,17
Cody Demps,14,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,14,14,14,14


In [16]:
df_games = df_all[lambda x: (x.name == "Team Totals") & (x.timetype == "TotalBasics")]

In [17]:
df_games.shape

(8612, 26)

In [18]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8612 entries, 13 to 829936
Data columns (total 26 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              8612 non-null   int64  
 1   name            8612 non-null   object 
 2   minutes_played  8612 non-null   float64
 3   fg              8612 non-null   float64
 4   fga             8612 non-null   float64
 5   fg_pct          8612 non-null   float64
 6   fg3             8612 non-null   float64
 7   fg3a            8612 non-null   float64
 8   fg3_pct         8612 non-null   float64
 9   ft              8612 non-null   float64
 10  fta             8612 non-null   float64
 11  ft_pct          8612 non-null   float64
 12  orb             8612 non-null   float64
 13  drb             8612 non-null   float64
 14  trb             8612 non-null   float64
 15  ast             8612 non-null   float64
 16  stl             8612 non-null   float64
 17  blk             8612 non-null 

In [29]:
df_games.groupby("game_code").count().sort_values("fg", ascending=False)[lambda x: x.id > 2]

,id,name,minutes_played,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,...,ast,stl,blk,tov,pf,pts,bpm,dnp,timetype,team
game_code,,,,,,,,,,,,,,,,,,,,,
201901210UTA,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
201902010UTA,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
201902020UTA,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
201902230UTA,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
201901140UTA,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
201902060UTA,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
201901230UTA,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
201903020UTA,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
201901180UTA,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4


In [30]:
dup_games = set(df_games.groupby("game_code").count().sort_values("fg", ascending=False)[lambda x: x.id > 2].index)

In [35]:
df_games[df_games.game_code.isin(dup_games)].sort_values(["game_code"])

,id,name,minutes_played,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,...,stl,blk,tov,pf,pts,bpm,dnp,timetype,team,game_code
56289,56290,Team Totals,240.0,37.0,93.0,0.398,16.0,42.0,0.381,18.0,...,7.0,6.0,16.0,16.0,108.0,0.0,0,TotalBasics,UTA,201812190UTA
59894,59895,Team Totals,240.0,37.0,93.0,0.398,16.0,42.0,0.381,18.0,...,7.0,6.0,16.0,16.0,108.0,0.0,0,TotalBasics,UTA,201812190UTA
56197,56198,Team Totals,240.0,38.0,95.0,0.400,10.0,31.0,0.323,17.0,...,10.0,9.0,12.0,21.0,103.0,0.0,0,TotalBasics,GSW,201812190UTA
59802,59803,Team Totals,240.0,38.0,95.0,0.400,10.0,31.0,0.323,17.0,...,10.0,9.0,12.0,21.0,103.0,0.0,0,TotalBasics,GSW,201812190UTA
58277,58278,Team Totals,240.0,42.0,89.0,0.472,9.0,25.0,0.360,14.0,...,11.0,5.0,17.0,21.0,107.0,0.0,0,TotalBasics,OKC,201812220UTA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57213,57214,Team Totals,240.0,38.0,92.0,0.413,11.0,34.0,0.324,24.0,...,9.0,12.0,16.0,23.0,111.0,0.0,0,TotalBasics,UTA,201902270UTA
61140,61141,Team Totals,240.0,37.0,96.0,0.385,15.0,39.0,0.385,22.0,...,6.0,10.0,15.0,22.0,111.0,0.0,0,TotalBasics,MIL,201903020UTA
56002,56003,Team Totals,240.0,37.0,96.0,0.385,15.0,39.0,0.385,22.0,...,6.0,10.0,15.0,22.0,111.0,0.0,0,TotalBasics,MIL,201903020UTA
56100,56101,Team Totals,240.0,44.0,96.0,0.458,12.0,29.0,0.414,15.0,...,13.0,7.0,13.0,23.0,115.0,0.0,0,TotalBasics,UTA,201903020UTA


In [37]:
df_games.drop_duplicates(["team", "game_code"], inplace=True)

C:\Users\Kyle\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [38]:
df_games.groupby("game_code").count()

,id,name,minutes_played,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,...,ast,stl,blk,tov,pf,pts,bpm,dnp,timetype,team
game_code,,,,,,,,,,,,,,,,,,,,,
201710170CLE,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
201710170GSW,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
201710180BOS,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
201710180DAL,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
201710180DET,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202103040NYK,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
202103040PHO,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
202103040POR,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2


In [39]:
df_games.head()

,id,name,minutes_played,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,...,stl,blk,tov,pf,pts,bpm,dnp,timetype,team,game_code
13,14,Team Totals,240.0,38.0,92.0,0.413,12.0,30.0,0.400,19.0,...,6.0,6.0,14.0,17.0,107.0,0.0,0,TotalBasics,DAL,201810260TOR
109,110,Team Totals,240.0,44.0,91.0,0.484,11.0,27.0,0.407,17.0,...,10.0,5.0,12.0,22.0,116.0,0.0,0,TotalBasics,TOR,201810260TOR
195,196,Team Totals,240.0,40.0,99.0,0.404,14.0,36.0,0.389,7.0,...,4.0,6.0,14.0,27.0,101.0,0.0,0,TotalBasics,BOS,201810190TOR
293,294,Team Totals,240.0,43.0,92.0,0.467,13.0,35.0,0.371,14.0,...,3.0,3.0,12.0,19.0,113.0,0.0,0,TotalBasics,TOR,201810190TOR
391,392,Team Totals,240.0,34.0,85.0,0.400,7.0,19.0,0.368,29.0,...,4.0,0.0,16.0,25.0,104.0,0.0,0,TotalBasics,CLE,201810170TOR


In [47]:
game_coder = re.compile("\d{8}")
home_team_regex = re.compile("[A-Z]{3}") 

In [57]:
d_Str = "202011030"

In [64]:
d_Str[4:6]

'11'

In [68]:
def parse_date(row):
    date = game_coder.search(row)[0]
    return dt.datetime(int(date[:4]), int(date[4:6]), int(date[6:]))

In [69]:
df_games = df_games.assign(date = lambda x: x.game_code.apply(parse_date))

In [70]:
def home_team(row):
    home = home_team_regex.search(row)[0]
    return home

In [72]:
df_games = df_games.assign(home_team = lambda x: x.game_code.apply(home_team))

In [75]:
df_games = df_games.assign(home_team_bin = lambda x: x.home_team == x.team)

In [76]:
df_adv = pd.read_sql("SELECT * FROM advanced_stats", con=conn)

In [83]:
df_adv = df_adv[df_adv.name == "Team Totals"]

In [86]:
df_adv = df_adv.drop_duplicates(["team", "game_code"])

In [87]:
df_adv.columns

Index(['id', 'name', 'minutes_played', 'ts_pct', 'efg_pct', 'fg3a_per_fga_pct',
       'fta_per_fga_pct', 'orb_pct', 'drb_pct', 'trb_pct', 'ast_pct',
       'stl_pct', 'blk_pct', 'tov_pct', 'usg_pct', 'off_rtg', 'def_rtg', 'bpm',
       'dnp', 'timetype', 'team', 'game_code'],
      dtype='object')

In [88]:
df_games.columns

Index(['id', 'name', 'minutes_played', 'fg', 'fga', 'fg_pct', 'fg3', 'fg3a',
       'fg3_pct', 'ft', 'fta', 'ft_pct', 'orb', 'drb', 'trb', 'ast', 'stl',
       'blk', 'tov', 'pf', 'pts', 'bpm', 'dnp', 'timetype', 'team',
       'game_code', 'date', 'home_team', 'home_team_bin'],
      dtype='object')

In [97]:
df_games = df_games.loc[:, ['team','minutes_played', 'fg', 'fga', 'fg_pct', 'fg3', 'fg3a',
       'fg3_pct', 'ft', 'fta', 'ft_pct', 'orb', 'drb', 'trb', 'ast', 'stl',
       'blk', 'tov', 'pf', 'pts', 'home_team', 'home_team_bin', 'date', 'game_code']]

In [96]:
df_adv = df_adv.loc[:, ['team', "game_code", 'ts_pct', 'efg_pct', 'fg3a_per_fga_pct',
       'fta_per_fga_pct', 'orb_pct', 'drb_pct', 'trb_pct', 'ast_pct',
       'stl_pct', 'blk_pct', 'tov_pct', 'usg_pct', 'off_rtg', 'def_rtg']]

In [98]:
df_games = df_games.reset_index(drop=True)
df_adv = df_adv.reset_index(drop=True)

In [101]:
df_full_stats = pd.merge(df_games, df_adv, on=["game_code", "team"])

In [106]:
df_full_stats = df_full_stats.loc[:, ['team', 'minutes_played', 'fg', 'fga', 'fg_pct', 'fg3', 'fg3a',
       'fg3_pct', 'ft', 'fta', 'ft_pct', 'orb', 'drb', 'trb', 'ast', 'stl',
       'blk', 'tov', 'ts_pct', 'efg_pct', 'fg3a_per_fga_pct', 'fta_per_fga_pct',
       'orb_pct', 'drb_pct', 'trb_pct', 'ast_pct', 'stl_pct', 'blk_pct',
       'tov_pct', 'usg_pct', 'off_rtg', 'def_rtg', 'pf','pts', 'home_team', 'home_team_bin', 'date', 'game_code']]

In [108]:
df_full_stats.groupby("home_team_bin").count()

,team,minutes_played,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,fta,...,blk_pct,tov_pct,usg_pct,off_rtg,def_rtg,pf,pts,home_team,date,game_code
home_team_bin,,,,,,,,,,,,,,,,,,,,,
False,4286,4286,4286,4286,4286,4286,4286,4286,4286,4286,...,4286,4286,4286,4286,4286,4286,4286,4286,4286,4286
True,4286,4286,4286,4286,4286,4286,4286,4286,4286,4286,...,4286,4286,4286,4286,4286,4286,4286,4286,4286,4286


In [109]:
home_teams_df = df_full_stats[df_full_stats.home_team_bin == True]
away_teams_df = df_full_stats[df_full_stats.home_team_bin != True]

In [ ]:
# create game-by-game dataframe 
# add seasons markers

In [110]:
home_teams_df.columns

Index(['team', 'minutes_played', 'fg', 'fga', 'fg_pct', 'fg3', 'fg3a',
       'fg3_pct', 'ft', 'fta', 'ft_pct', 'orb', 'drb', 'trb', 'ast', 'stl',
       'blk', 'tov', 'ts_pct', 'efg_pct', 'fg3a_per_fga_pct',
       'fta_per_fga_pct', 'orb_pct', 'drb_pct', 'trb_pct', 'ast_pct',
       'stl_pct', 'blk_pct', 'tov_pct', 'usg_pct', 'off_rtg', 'def_rtg', 'pf',
       'pts', 'home_team', 'home_team_bin', 'date', 'game_code'],
      dtype='object')

In [111]:
away_teams_df.columns

Index(['team', 'minutes_played', 'fg', 'fga', 'fg_pct', 'fg3', 'fg3a',
       'fg3_pct', 'ft', 'fta', 'ft_pct', 'orb', 'drb', 'trb', 'ast', 'stl',
       'blk', 'tov', 'ts_pct', 'efg_pct', 'fg3a_per_fga_pct',
       'fta_per_fga_pct', 'orb_pct', 'drb_pct', 'trb_pct', 'ast_pct',
       'stl_pct', 'blk_pct', 'tov_pct', 'usg_pct', 'off_rtg', 'def_rtg', 'pf',
       'pts', 'home_team', 'home_team_bin', 'date', 'game_code'],
      dtype='object')

In [114]:
g_by_g_df = pd.merge(home_teams_df, away_teams_df, on=["game_code"], suffixes=("_home", "_away"))

In [124]:
g_by_g_df = g_by_g_df.loc[:, ['team_home', 'team_away', 'minutes_played_home', 'fg_home', 'fga_home',
       'fg_pct_home', 'fg3_home', 'fg3a_home', 'fg3_pct_home', 'ft_home',
       'fta_home', 'ft_pct_home', 'orb_home', 'drb_home', 'trb_home',
       'ast_home', 'stl_home', 'blk_home', 'tov_home', 'ts_pct_home',
       'efg_pct_home', 'fg3a_per_fga_pct_home', 'fta_per_fga_pct_home',
       'orb_pct_home', 'drb_pct_home', 'trb_pct_home', 'ast_pct_home',
       'stl_pct_home', 'blk_pct_home', 'tov_pct_home', 'usg_pct_home',
       'off_rtg_home', 'def_rtg_home', 'pf_home', 'pts_home', 'home_team_home',
       'home_team_bin_home',       
       'minutes_played_away', 'fg_away', 'fga_away', 'fg_pct_away', 'fg3_away',
       'fg3a_away', 'fg3_pct_away', 'ft_away', 'fta_away', 'ft_pct_away',
       'orb_away', 'drb_away', 'trb_away', 'ast_away', 'stl_away', 'blk_away',
       'tov_away', 'ts_pct_away', 'efg_pct_away', 'fg3a_per_fga_pct_away',
       'fta_per_fga_pct_away', 'orb_pct_away', 'drb_pct_away', 'trb_pct_away',
       'ast_pct_away', 'stl_pct_away', 'blk_pct_away', 'tov_pct_away',
       'usg_pct_away', 'off_rtg_away', 'def_rtg_away', 'pf_away', 'pts_away',
       'home_team_away', 'home_team_bin_away', 'game_code', 'date_home']]

In [126]:
g_by_g_df = g_by_g_df.rename(columns={"date_home": "date"})

In [ ]:
# Oct 17, 2017 - June 8, 2018
# October 16, 2018 - June 13, 2019
# October 22, 2019 - October 11, 2020
# Dec 22 - March 7th 2021

In [127]:
def seasoner(date):
    if date <= dt.datetime(2018, 6, 8):
        return "17-18"
    elif date <= dt.datetime(2019, 6, 13):
        return "18-19"
    elif date <= dt.datetime(2020, 10, 11):
        return "19-20"
    else:
        return "20-21"

In [130]:
g_by_g_df.assign(NBA_season = lambda x: x.date.apply(seasoner)).groupby("NBA_season").count()

,team_home,team_away,minutes_played_home,fg_home,fga_home,fg_pct_home,fg3_home,fg3a_home,fg3_pct_home,ft_home,...,tov_pct_away,usg_pct_away,off_rtg_away,def_rtg_away,pf_away,pts_away,home_team_away,home_team_bin_away,game_code,date
NBA_season,,,,,,,,,,,,,,,,,,,,,
17-18,1312,1312,1312,1312,1312,1312,1312,1312,1312,1312,...,1312,1312,1312,1312,1312,1312,1312,1312,1312,1312
18-19,1312,1312,1312,1312,1312,1312,1312,1312,1312,1312,...,1312,1312,1312,1312,1312,1312,1312,1312,1312,1312
19-20,1142,1142,1142,1142,1142,1142,1142,1142,1142,1142,...,1142,1142,1142,1142,1142,1142,1142,1142,1142,1142
20-21,520,520,520,520,520,520,520,520,520,520,...,520,520,520,520,520,520,520,520,520,520


In [131]:
g_by_g_df = g_by_g_df.assign(NBA_season = lambda x: x.date.apply(seasoner))

In [132]:
def playoffs(date):
    # 4/11/18
    if (date >= dt.datetime(2018, 4, 11)) and (date <= dt.datetime(2018, 6, 8)):
        return 1
    # 4/13/19
    elif (date >= dt.datetime(2019, 4, 13)) and (date <= dt.datetime(2019, 6, 13)):
        return 1
    # 8/17/20
    elif (date >= dt.datetime(2020, 8, 15)) and (date <= dt.datetime(2020, 10, 11)):
        return 1
    else:
        return 0

In [135]:
g_by_g_df = g_by_g_df.assign(playoffs = lambda x: x.date.apply(playoffs))

In [138]:
# Finished pairing Down the Data to a Game by Game table structured by each home team
g_by_g_df.to_csv("../data/interim/game_by_game_stats.csv", index=False)

In [146]:
# Next: Pair Bookmaker Data With Right Games:
odds_17 = pd.read_csv("../data/external/vegas_17_18.csv")
odds_17_playoffs = pd.read_csv("../data/external/vegas_playoff_17_18.csv")
odds_18 = pd.read_csv("../data/external/vegas_18_19.csv")
odds_18_playoffs = pd.read_csv("../data/external/vegas_playoff_18_19.csv")

In [141]:
odds_17.columns

Index(['Date', 'Location', 'Team', 'OppTeam', 'TeamId', 'GameId',
       'PercentBet_ML', 'Open_Line_ML', 'Pinnacle_ML', '5dimes_ML',
       'Heritage_ML', 'Bovada_ML', 'Betonline_ML', 'Average_Line_ML',
       'Best_Line_ML', 'Worst_Line_ML', 'PercentBet_Spread',
       'Open_Line_Spread', 'Open_Odds_Spread', 'Pinnacle_Line_Spread',
       'Pinnacle_Odds_Spread', '5dimes_Line_Spread', '5dimes_Odds_Spread',
       'Heritage_Line_Spread', 'Heritage_Odds_Spread', 'Bovada_Line_Spread',
       'Bovada_Odds_Spread', 'Betonline_Line_Spread', 'Betonline_Odds_Spread',
       'Average_Line_Spread', 'Average_Odds_Spread', 'Best_Line_Spread',
       'Worst_Line_Spread', 'Best_Odds_Spread', 'Worst_Odds_Spread',
       'PercentBet_OU', 'Open_Line_OU', 'Open_Odds_OU', 'Pinnacle_Line_OU',
       'Pinnacle_Odds_OU', '5dimes_Line_OU', '5dimes_Odds_OU',
       'Heritage_Line_OU', 'Heritage_Odds_OU', 'Bovada_Line_OU',
       'Bovada_Odds_OU', 'Betonline_Line_OU', 'Betonline_Odds_OU',
       'Average_Line

In [144]:
odds_17_playoffs.head()

,Date,Location,Team,OppTeam,TeamId,GameId,PercentBet_ML,Open_Line_ML,Pinnacle_ML,5dimes_ML,...,Average_Line_OU,Average_Odds_OU,Best_Line_OU,Worst_Line_OU,Best_Odds_OU,Worst_Odds_OU,Pts,Spread,Result,Total
0,4/14/2018,away,Washington,Toronto,1610612764,41700101,55.8,315,324,320,...,213.33,-108.83,214.5,209.5,-105,-115,106,-8,L,220
1,4/14/2018,home,Toronto,Washington,1610612761,41700101,44.2,-375,-373,-355,...,213.33,-106.17,214.5,209.5,-102,-110,114,8,W,220
2,4/14/2018,away,Miami,Philadelphia,1610612748,41700121,50.3,230,208,210,...,214.08,-105.50,214.5,214.0,-101,-110,103,-27,L,233
3,4/14/2018,home,Philadelphia,Miami,1610612755,41700121,49.7,-275,-233,-230,...,214.08,-110.33,214.5,214.0,-105,-115,130,27,W,233
4,4/14/2018,away,San Antonio,Golden State,1610612759,41700151,49.8,300,330,340,...,206.25,-108.33,210.5,205.0,-105,-110,92,-21,L,205


In [145]:
odds_17.columns == odds_17_playoffs.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

In [147]:
odds_17.loc[:, ["playoffs"]] = 0
odds_18.loc[:, ["playoffs"]] = 0
odds_17_playoffs.loc[:, ["playoffs"]] = 1
odds_18_playoffs.loc[:, ["playoffs"]] = 1

In [149]:
full_odds_df = pd.concat([odds_17, odds_17_playoffs, odds_18, odds_18_playoffs], axis=0, ignore_index=True)

In [150]:
full_odds_df.to_csv("../data/interim/vegas_data.csv")

In [ ]:
# Remaining Objectives:
 # Get the Vegas Data on a game by game level as is the current case with g_by_g_df
    # Do this by constructing the high level information in the full odds dataframe to reverse back into the original game_codes we have today
    # I think we can still do this by joining on team and date as well. Which should be easier and quicker.
 # Once we have this we will end up with Sportline information for all games of the the past two full seasons (17-18) and (18-19)
    # I did this first so we could have additional information about the gambling markets to compare to the model we will create (i.e.: A baseline to compare our error to the true market error for each game)

    # So once I get this data to a game by game configuration, I will able to match it with our current game information and start the model building process.  